# The Next Hokage (GUI)

### Helper Function

In [1]:
def check_hand_sign(results):
    """Plot the frame and get the hand sign detected."""
    for result in results:
        frame = result.plot()
        if len(result.boxes.cls) == 1:
            hand_sign = result.names[int(result.boxes.cls)]
        else:
            hand_sign = None
    
    return frame, hand_sign

In [2]:
import random

def deploy_damage(hand_sign, player_list, player_id, dummy=None):
    """Deploy Jutsu damage on the opponent."""
    player = player_list[player_id-1]
    
    if dummy:
        other_player_id = 3
    else:
        other_player_id = 2 if player_id == 1 else 1
        
    other_player = player_list[other_player_id-1]
    defeated = False
    if hand_sign is not None:
        for jutsu in player.jutsu_list:
            jutsu.check_pattern(hand_sign, player_id)
            if jutsu.detected:
                print(f'Player {player_id} deployed {jutsu.name}!')
                playVideo(jutsu.video_path)
                
                # 20% damage variation
                damage = int(jutsu.damage*random.uniform(0.8, 1.2))
                other_player.health -= damage
                
                if dummy:
                    other_player_name = f'Dummy player'
                else:
                    other_player_name = f'Player {other_player_id}'
                
                if other_player.health < 0:
                    print(f'Player {player_id} defeated {other_player_name}!')
                    defeated = True
                else:
                    print(f'{other_player_name} suffered {damage} damage and left {other_player.health} hp.\n')
                    
                player.resetJutsu()
                break
    
    return defeated

In [3]:
import cv2
# ffpyplayer for playing audio
from ffpyplayer.player import MediaPlayer

def playVideo(video_path):
    """Play video from specified video path."""
    video = cv2.VideoCapture(video_path)
    player = MediaPlayer(video_path)
    while True:
        grabbed, frame = video.read()
        audio_frame, val = player.get_frame()
        if not grabbed:
            # print("End of video")
            break
        if cv2.waitKey(28) & 0xFF == ord("q"):
            break
        cv2.imshow("Video", frame)
        if val != 'eof' and audio_frame is not None:
            # audio
            img, t = audio_frame
    video.release()
    cv2.destroyAllWindows()

In [4]:
jutsu_list = []
    
class Jutsu:
    """
    A base class for creating Jutsu.
    
    Attributes:
        name (str): Name of Jutsu technique
        success (list): Jutsu technique pattern.
        damage (int): Amount of damage.
        video_path (Path): Path to the Jutsu animation.
        hand_signs_detected (list): List of successful Jutsu signs.
        detected (bool): Flag to enable video playing.
    """
    
    def __init__(self, name, success, damage, video_path):
        """
        Initializes the Jutsu class.

        Args:
            name (str): Name of Jutsu technique
            success (list): Jutsu technique pattern.
            damage (int): Amount of damage.
            video_path (Path): Path to the Jutsu animation.
        """
        jutsu_list.append(self)
        self.name = name
        self.success = success
        self.damage = damage
        self.video_path = video_path
        self.hand_signs_detected = []
        self.detected = False
    
    def check_pattern(self, hand_sign, player_id):
        """Validate Jutsu pattern."""
        if len(self.hand_signs_detected) == len(self.success):
            self.detected = True
        elif hand_sign == self.success[len(self.hand_signs_detected)]:
            self.hand_signs_detected.append(hand_sign)
            print(f'{hand_sign.title()} detected for Player {player_id}.')

In [5]:
import copy

class Player:
    """
    A base class for creating Jutsu.
    
    Attributes:
        jutsu_list (list): All Jutsu technique patterns.
        health (int): Player health.
    """
    
    def __init__(self, jutsu_list):
        """
        Initializes the Player class.

        Args:
            jutsu_list (list): All Jutsu technique patterns.
        """
        self.jutsu_list = copy.deepcopy(jutsu_list)
        self.health = 1000

    def resetJutsu(self):
        """Reset lists of successful Jutsu signs."""
        for jutsu in self.jutsu_list:
            jutsu.hand_signs_detected = []
            jutsu.detected = False

### Main Code

Create three Justu as examples for the game.

In [6]:
import os 

name = 'Summoning Jutsu'
success = ['boar', 'dog', 'bird', 'monkey', 'ram']
damage = 200
video_path = os.getcwd() + "\\animation\\summoning_jutsu_vid.mp4"
summoningJutsu = Jutsu(name, success, damage, video_path)

name = 'Fireball Jutsu'
success = ['horse', 'tiger', 'snake', 'ram', 'monkey', 'boar', 'horse', 'tiger']
damage = 300
video_path = os.getcwd() + "\\animation\\fireball_jutsu_vid.mp4"
fireballJutsu = Jutsu(name, success, damage, video_path)

name = 'Chidori Jutsu'
success = ['ox', 'hare', 'monkey']
damage = 100
video_path = os.getcwd() + "\\animation\\chidori_jutsu_vid.mp4"
chidoriJutsu = Jutsu(name, success, damage, video_path)

### Player

Create two players and a training player for the game.

In [7]:
player1 = Player(jutsu_list)
player2 = Player(jutsu_list)
training_player = Player(jutsu_list)
player_list = [player1, player2, training_player]

### Start the game!

Instruction: Player has to display specific Jutsu technique pattern to deal damage. The Jutsu technique is described on the start of the game. Once the Jutsu technique is accomplised, an animation will be displayed as well as the damage done.

In [8]:
import tkinter as tk
import os
from ultralytics import YOLO
import cv2

model = YOLO('best.pt')

class MainMenu:
    def __init__(self, root):
        self.root = root
        self.root.title("The Next Hokage")
        self.root.geometry("700x500")
        
        # Load the image
        path = os.getcwd() + "\\animation\\bg.png"
        self.background_image = tk.PhotoImage(file=path)

        # Create a label to hold the image
        self.background_label = tk.Label(self.root, image=self.background_image)
        self.background_label.place(x=0, y=0, relwidth=1, relheight=1)
        
        self.create_widgets()

    def create_widgets(self):
        # Adding title
        title_label = tk.Label(self.root, text="The Next Hokage", font=("Comic Sans MS", 40, "bold"), bg="white", fg="#FF8C00")
        title_label.pack(pady=20)  # Padding above the title label
        
        frame = tk.Frame(self.root)
        frame.pack(expand=True)  # Expanding the frame to fill the window
        
        # Creating blank space (padding) above the buttons
        tk.Label(frame, text="Selections", font=("Comic Sans MS", 25, "bold"), bg="white", fg="black").pack()  # Blank label as padding
        
        self.create_button(frame, "Training", self.option_1_selected, width=15, font=("Comic Sans MS", 12, 'bold'), fg="white", bg="#0080FE")
        self.create_button(frame, "PvP", self.option_2_selected, width=15, font=("Comic Sans MS", 12, 'bold'), fg="white", bg="#0080FE")
        self.create_button(frame, "Exit", self.option_3_selected, width=15, font=("Comic Sans MS", 12, 'bold'), fg="white", bg="#0080FE")

    def create_button(self, frame, text, command, **kwargs):
        button = tk.Button(frame, text=text, command=command, **kwargs)
        button.pack(side='top', pady=5)

    def option_1_selected(self):
        # print("Option 1 selected")
        
        # start webcam
        cap = cv2.VideoCapture(0)
        cap.set(3, 640)
        cap.set(4, 480)

        # threshold
        thres = 0.4

        # display instructions
        print('Welcome to The Next Hokage!')
        print('\nInstruction: Player has to display specific Jutsu technique pattern to deal damage.')
        print('Jutsu Techniques:')
        for jutsu in jutsu_list:
            print(f'{jutsu.name}: {", ".join(jutsu.success)}')
        print()

        while True:
            success, img = cap.read()
            
            # vertically flip image
            training_frame = cv2.flip(img, 1)
            
            results_training = model(training_frame, stream=True, verbose=False) # conf=thres

            training_frame, hand_sign_training = check_hand_sign(results_training)
            cv2.imshow('Webcam', training_frame)

            defeated = deploy_damage(hand_sign_training, player_list, 1, dummy=True)
                        
            if cv2.waitKey(1) == ord('q'):
                print('Game exit.')
                break

        cap.release()
        cv2.destroyAllWindows()

    def option_2_selected(self):
        # print("Option 2 selected")
        
        # start webcam
        cap = cv2.VideoCapture(0)
        cap.set(3, 640)
        cap.set(4, 480)

        # threshold
        thres = 0.4

        # display instructions
        print('Welcome to The Next Hokage!')
        print('\nInstruction: Player has to display specific Jutsu technique pattern to deal damage.')
        print('Jutsu Techniques:')
        for jutsu in jutsu_list:
            print(f'{jutsu.name}: {", ".join(jutsu.success)}')
        print()
        
        while True:
            success, img = cap.read()
            
            # vertically flip image
            img = cv2.flip(img, 1)
            left_frame = img[:, :img.shape[1]//2]
            right_frame = img[:, img.shape[1]//2:]
            
            results_player1 = model(left_frame, stream=True, verbose=False) # conf=thres
            results_player2 = model(right_frame, stream=True, verbose=False)
            left_frame, hand_sign_player1 = check_hand_sign(results_player1)
            right_frame, hand_sign_player2 = check_hand_sign(results_player2)
                    
            cv2.imshow('Player1', left_frame)
            cv2.imshow('Player2', right_frame)
            
            defeated1 = deploy_damage(hand_sign_player1, player_list, 1)
            defeated2 = deploy_damage(hand_sign_player2, player_list, 2)
            
            if cv2.waitKey(1) == ord('q') or defeated1 or defeated2:
                print('Game exit.')
                break

        cap.release()
        cv2.destroyAllWindows()

    # exit function
    def option_3_selected(self):
        print("'Exit' selected")
        self.root.destroy()  # Close the Tkinter window
        cv2.destroyAllWindows()

def main():
    root = tk.Tk()
    app = MainMenu(root)
    root.mainloop()

if __name__ == "__main__":
    main()


Welcome to The Next Hokage!

Instruction: Player has to display specific Jutsu technique pattern to deal damage.
Jutsu Techniques:
Summoning Jutsu: boar, dog, bird, monkey, ram
Fireball Jutsu: horse, tiger, snake, ram, monkey, boar, horse, tiger
Chidori Jutsu: ox, hare, monkey

Horse detected for Player 1.
Boar detected for Player 1.
Ox detected for Player 1.
Dog detected for Player 1.
Bird detected for Player 1.
Monkey detected for Player 1.
Ram detected for Player 1.
Player 1 deployed Summoning Jutsu!
dummy player suffered 221 damage and left 779 hp.

Game exit.
'Exit' selected
